In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

%matplotlib inline

### EDA

In [3]:
gc_members = pd.read_csv('../Data Files/NSS_GreatCare_Members.csv')
admissions = pd.read_csv('../Data Files/NSS_Hospital_Inpatient_Admissions.csv')
pc_visits = pd.read_csv('../Data Files/NSS_PrimaryCare_Office_Visits.csv')

C:\Users\darcy\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (3,5,17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
admissions = admissions.astype({'Attending Provider NPI':object, 
                                'Encounter Start Date':'datetime64[ns]',
                                'Encounter End Date':'datetime64[ns]',
                                'CMS Admit Type Code':object,
                                'CMS Patient Discharge Status Code':object,
                                'CMS Place of Service Code':object})
pc_visits = pc_visits.astype({'Attending Provider NPI':object, 
                              'Facility Provider NPI':object, 
                              'Service Provider NPI':object, 
                              'Provider Group NPI':object,
                              'Encounter Date':'datetime64[ns]'})
gc_members = gc_members.astype({'Patient Date of Birth':'datetime64[ns]',
                                'Patient ZIP Code 5-Digit':object,
                                'Administratively Assigned PCP Provider NPI':object,
                                'Empirically Attributed PCP Individual Provider NPI':object,
                                'Empirically Attributed PCP Group Provider NPI':object,
                                'Primary Payor Description':object,
                                'CMS Dual Status Code':object,
                                'Period Start Date':'datetime64[ns]',
                                'Period End Date':'datetime64[ns]'})

In [5]:
admissions.head()

,Encounter ID,Patient ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Facility Provider ID,Attending Provider ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,...,CMS Admit Source Code,CMS Patient Discharge Status Code,CMS Place of Service Code,Principal Discharge Diagnosis ICD-9-CM Code,Principal Discharge Diagnosis ICD-9-CM Description,Principal Discharge Diagnosis ICD-10-CM Code,Principal Discharge Diagnosis ICD-10-CM Description,Primary Plan Allowed Amount,Primary Plan Paid Amount,Patient Paid Amount
0,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,DS1|1,DS1|5047W,DS1|NPI|1306890389,DS1|NPI|1417170523,1.41717e+09,1306890389,2013-01-12,2013-01-14,...,1,1,21,NaN,NaN,O70.0,First degree perineal laceration during delivery,0,6620.72,250.0
1,HIA|DS1|10043|DS1|NPI|1902803315|1,DS1|10043,DS1|1,DS1|5047Y,DS1|NPI|1902803315,DS1|NPI|1669599197,1.6696e+09,1902803315,2014-01-05,2014-01-12,...,1,1,21,348.4,Compression of brain,NaN,NaN,0,46465.40,250.0
2,HIA|DS1|10073|DS1|NPI|1902803315|1,DS1|10073,DS1|1,DS1|5047X,DS1|NPI|1902803315,DS1|NPI|1407880511,1.40788e+09,1902803315,2012-06-13,2012-06-21,...,1,1,21,NaN,NaN,K91.71,Accidental puncture and laceration of a digest...,0,43879.13,250.0
3,HIA|DS1|10101|DS1|NPI|1427055839|1,DS1|10101,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1407008907,1.40701e+09,1427055839,2012-12-06,2012-12-09,...,1,1,21,NaN,NaN,O70.1,Second degree perineal laceration during delivery,0,14800.00,250.0
4,HIA|DS1|10118|DS1|NPI|1992818256|1,DS1|10118,DS1|1,DS1|5047W,DS1|NPI|1992818256,DS1|NPI|1518045608,1.51805e+09,1992818256,2015-03-11,2015-03-14,...,2,1,NaN,NaN,NaN,F32.9,"Major depressive disorder, single episode, uns...",0,0.00,0.0


In [6]:
admissions.shape

(6213, 23)

In [7]:
admissions.columns

Index(['Encounter ID', 'Patient ID', 'Encounter Primary Payor ID',
       'Encounter Primary Plan ID', 'Facility Provider ID',
       'Attending Provider ID', 'Attending Provider NPI',
       'Facility Provider NPI', 'Encounter Start Date', 'Encounter End Date',
       'Is Emergency Department Visit Parent Encounter',
       'Is Observation Stay Parent Encounter', 'CMS Admit Type Code',
       'CMS Admit Source Code', 'CMS Patient Discharge Status Code',
       'CMS Place of Service Code',
       'Principal Discharge Diagnosis ICD-9-CM Code',
       'Principal Discharge Diagnosis ICD-9-CM Description',
       'Principal Discharge Diagnosis ICD-10-CM Code',
       'Principal Discharge Diagnosis ICD-10-CM Description',
       'Primary Plan Allowed Amount', 'Primary Plan Paid Amount',
       'Patient Paid Amount'],
      dtype='object')

In [8]:
pc_visits.head()

,Encounter ID,Patient ID,Service Provider ID,Attending Provider ID,Provider Group Provider ID,Facility Provider ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Attending Provider NPI,Facility Provider NPI,Service Provider NPI,Provider Group NPI,Encounter Date,Is Qualifying Preventive Primary Care Service,Is Qualifying Primary Care Service Provider,Is Qualifying Primary Care Attending,Is Qualifying Primary Care Provider Group,Qualifying Preventive Primary Care HCPCS Code
0,COV|EE|DS1|10076|20110225|DS1|NPI|1487856779,DS1|10076,DS1|NPI|1487856779,NaN,DS1|NPI|1043427248,NaN,DS1|1,DS1|5047W,NaN,NaN,1.48786e+09,1.04343e+09,2011-02-18,0,1,0,0,NaN
1,COV|EE|DS1|10088|20140222|DS1|NPI|1528030996,DS1|10088,DS1|NPI|1528030996,DS1|NPI|1528030996,DS1|NPI|1043427248,DS1|NPI|1902803315,DS1|1,DS1|5047W,1.52803e+09,1.9028e+09,1.52803e+09,1.04343e+09,2014-02-22,0,1,1,1,NaN
2,COV|EE|DS1|10165|20140907|DS1|NPI|1477579365,DS1|10165,DS1|NPI|1477579365,NaN,DS1|NPI|1043427248,NaN,DS1|1,DS1|5047W,NaN,NaN,1.47758e+09,1.04343e+09,2014-08-31,0,1,0,0,NaN
3,COV|EE|DS1|10240|20130624|DS1|NPI|1821178088,DS1|10240,DS1|NPI|1821178088,DS1|NPI|1821178088,DS1|NPI|1407058787,DS1|NPI|1104842020,DS1|1,DS1|5047W,1.82118e+09,1.10484e+09,1.82118e+09,1.40706e+09,2013-06-17,0,1,1,1,NaN
4,COV|EE|DS1|10293|20160904|DS1|NPI|1609004191,DS1|10293,DS1|NPI|1609004191,NaN,DS1|NPI|1184722779,NaN,DS1|1,DS1|5047X,NaN,NaN,1.609e+09,1.18472e+09,2016-09-04,0,1,0,0,NaN


In [9]:
gc_members.head()

,Patient ID,Primary Plan ID,Primary Payor ID,Is Patient Alive,Patient Age Integer,Patient Date of Birth,Patient ZIP Code 5-Digit,Count Primary Care Visits in Last 36 Months,Count Visits with Attributee Individual Primary Care Provider in Last 36 Months,Count Visits with Attributee Primary Care Provider Group in Last 36 Months,...,Is CCW Prostate Cancer,Is CCW Rheumatoid Arthritis or Osteoarthritis,Is CCW Stroke or Transient Ischemic Attack,Is CCW ASCVD,Is CCW Anxiety Disorders,Is CCW Depressive Disorders,Is CCW Schizophrenia or Other Psychotic Disorders,Is CCW Bipolar Disorder,Period Start Date,Period End Date
0,DS1|1,DS1|5047W,DS1|1,1,54,1958-11-08,92111,0,NaN,NaN,...,0,0,0,0,0,0,0,0,2013-01-01,2013-04-11
1,DS1|10,DS1|5047W,DS1|1,1,54,1958-12-10,90024,6,1.0,5.0,...,0,0,0,0,1,0,0,0,2012-12-10,2013-01-01
2,DS1|100,DS1|5047W,DS1|1,1,2,2012-08-04,90008,1,1.0,1.0,...,0,0,0,0,0,0,0,0,2015-04-18,2015-05-18
3,DS1|1000,DS1|5047W,DS1|1,1,6,2003-12-25,94131,0,NaN,NaN,...,0,0,0,0,0,0,0,0,2010-01-01,2010-04-09
4,DS1|10000,DS1|5047W,DS1|1,1,7,2005-07-15,90301,0,NaN,NaN,...,0,0,0,0,0,0,0,0,2012-08-01,2012-10-02


In [10]:
gc_members.columns

Index(['Patient ID', 'Primary Plan ID', 'Primary Payor ID', 'Is Patient Alive',
       'Patient Age Integer', 'Patient Date of Birth',
       'Patient ZIP Code 5-Digit',
       'Count Primary Care Visits in Last 36 Months',
       'Count Visits with Attributee Individual Primary Care Provider in Last 36 Months',
       'Count Visits with Attributee Primary Care Provider Group in Last 36 Months',
       'Administratively Assigned PCP Provider NPI',
       'Empirically Attributed PCP Individual Provider NPI',
       'Empirically Attributed PCP Individual Provider Description',
       'Empirically Attributed PCP Group Provider NPI',
       'Empirically Attributed PCP Group Provider Description',
       'Primary Plan Description', 'Primary Payor Description',
       'Primary Plan Financial Class Description',
       'Is Primary Plan Financial Class Commercial',
       'Is Primary Plan Financial Class Medicare FFS',
       'Is Primary Plan Financial Class Medicare Advantage',
       'Is Pri

### 1. What is our 30-day all cause readmission rate across hospitals our members are admitted to?

For each index event that each patient has, get only the **first** readmission within the 30 days following that index event.

In [11]:
admissions = admissions.sort_values(['Patient ID', 'Encounter Start Date']).reset_index(drop=True)
admissions

,Encounter ID,Patient ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Facility Provider ID,Attending Provider ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,...,CMS Admit Source Code,CMS Patient Discharge Status Code,CMS Place of Service Code,Principal Discharge Diagnosis ICD-9-CM Code,Principal Discharge Diagnosis ICD-9-CM Description,Principal Discharge Diagnosis ICD-10-CM Code,Principal Discharge Diagnosis ICD-10-CM Description,Primary Plan Allowed Amount,Primary Plan Paid Amount,Patient Paid Amount
0,HIA|DS1|10|DS1|NPI|1710918545|1,DS1|10,DS1|1,DS1|5047W,DS1|NPI|1710918545,DS1|NPI|1962461012,1.96246e+09,1710918545,2013-06-13,2013-06-16,...,1,1,21,NaN,NaN,K35.2,Acute appendicitis with generalized peritonitis,0,29219.00,250.0
1,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,DS1|1,DS1|5047W,DS1|NPI|1306890389,DS1|NPI|1417170523,1.41717e+09,1306890389,2013-01-12,2013-01-14,...,1,1,21,NaN,NaN,O70.0,First degree perineal laceration during delivery,0,6620.72,250.0
2,HIA|DS1|10043|DS1|NPI|1902803315|1,DS1|10043,DS1|1,DS1|5047Y,DS1|NPI|1902803315,DS1|NPI|1669599197,1.6696e+09,1902803315,2014-01-05,2014-01-12,...,1,1,21,348.4,Compression of brain,NaN,NaN,0,46465.40,250.0
3,HIA|DS1|10073|DS1|NPI|1902803315|1,DS1|10073,DS1|1,DS1|5047X,DS1|NPI|1902803315,DS1|NPI|1407880511,1.40788e+09,1902803315,2012-06-13,2012-06-21,...,1,1,21,NaN,NaN,K91.71,Accidental puncture and laceration of a digest...,0,43879.13,250.0
4,HIA|DS1|10101|DS1|NPI|1427055839|1,DS1|10101,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1407008907,1.40701e+09,1427055839,2012-12-06,2012-12-09,...,1,1,21,NaN,NaN,O70.1,Second degree perineal laceration during delivery,0,14800.00,250.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6208,HIA|DS1|9979|DS1|NPI|1689772592|1,DS1|9979,DS1|1,DS1|5047X,DS1|NPI|1689772592,DS1|NPI|1720043417,1.72004e+09,1689772592,2015-02-06,2015-02-11,...,2,6,21,NaN,NaN,O44.13,"Complete placenta previa with hemorrhage, thir...",0,23598.00,250.0
6209,HIA|DS1|9981|DS1|NPI|1427055839|1,DS1|9981,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1215968474,1.21597e+09,1427055839,2014-08-07,2014-08-10,...,1,1,21,664.11,"Second-degree perineal laceration, delivered, ...",NaN,NaN,0,14800.00,250.0
6210,HIA|DS1|9994|DS1|NPI|1689608150|1,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1861831836,1.86183e+09,1689608150,2015-11-21,2015-11-23,...,1,1,21,NaN,NaN,O76,Abnormality in fetal heart rate and rhythm com...,0,17919.75,250.0
6211,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1750544516,1.75054e+09,1689608150,2015-11-24,2015-11-27,...,1,1,21,NaN,NaN,O42.92,"Full-term premature rupture of membranes, unsp...",0,19919.42,250.0


In [12]:
admissions = admissions[['Patient ID','Encounter Start Date','Encounter End Date']]
admissions

,Patient ID,Encounter Start Date,Encounter End Date
0,DS1|10,2013-06-13,2013-06-16
1,DS1|10033,2013-01-12,2013-01-14
2,DS1|10043,2014-01-05,2014-01-12
3,DS1|10073,2012-06-13,2012-06-21
4,DS1|10101,2012-12-06,2012-12-09
...,...,...,...
6208,DS1|9979,2015-02-06,2015-02-11
6209,DS1|9981,2014-08-07,2014-08-10
6210,DS1|9994,2015-11-21,2015-11-23
6211,DS1|9994,2015-11-24,2015-11-27


In [13]:
admission_counts = admissions['Patient ID'].value_counts()
mult_admissions = admissions[admissions['Patient ID'].isin(admission_counts.index[admission_counts.gt(1)])]
mult_admissions = mult_admissions.reset_index(drop=True)
mult_admissions

,Patient ID,Encounter Start Date,Encounter End Date
0,DS1|10132,2013-03-30,2013-04-01
1,DS1|10132,2013-08-14,2013-08-16
2,DS1|10253,2010-04-02,2010-04-16
3,DS1|10253,2010-06-02,2010-06-06
4,DS1|10253,2011-10-15,2011-11-13
...,...,...,...
2377,DS1|9943,2013-10-01,2013-10-16
2378,DS1|9943,2013-10-25,2013-10-28
2379,DS1|9943,2014-10-28,2014-10-30
2380,DS1|9994,2015-11-21,2015-11-23


In [15]:
mult_admissions['30_after_end'] = mult_admissions['Encounter End Date'] + datetime.timedelta(30)
mult_admissions['30_after_end'] = mult_admissions.groupby('Patient ID')['30_after_end'].shift(1)
mult_admissions.head(10)

,Patient ID,Encounter Start Date,Encounter End Date,30_after_end
0,DS1|10132,2013-03-30,2013-04-01,NaT
1,DS1|10132,2013-08-14,2013-08-16,2013-05-01
2,DS1|10253,2010-04-02,2010-04-16,NaT
3,DS1|10253,2010-06-02,2010-06-06,2010-05-16
4,DS1|10253,2011-10-15,2011-11-13,2010-07-06
5,DS1|10291,2010-05-14,2010-05-15,NaT
6,DS1|10291,2011-02-24,2011-03-05,2010-06-14
7,DS1|10451,2013-03-20,2013-03-23,NaT
8,DS1|10451,2013-03-27,2013-04-16,2013-04-22
9,DS1|10451,2013-04-16,2013-04-26,2013-05-16


In [18]:
example = mult_admissions.loc[mult_admissions['Patient ID']=='DS1|10451']
example

,Patient ID,Encounter Start Date,Encounter End Date,30_after_end
7,DS1|10451,2013-03-20,2013-03-23,NaT
8,DS1|10451,2013-03-27,2013-04-16,2013-04-22
9,DS1|10451,2013-04-16,2013-04-26,2013-05-16
10,DS1|10451,2013-05-01,2013-05-09,2013-05-26
11,DS1|10451,2013-06-25,2013-07-01,2013-06-08
12,DS1|10451,2013-07-26,2013-08-03,2013-07-31
13,DS1|10451,2013-08-31,2013-09-02,2013-09-02
14,DS1|10451,2013-11-14,2013-11-20,2013-10-02
15,DS1|10451,2013-12-17,2013-12-19,2013-12-20
16,DS1|10451,2014-04-24,2014-04-26,2014-01-18


In [41]:
example_dict = pd.Series(example['Encounter End Date'].values,index=example['Patient ID']).to_dict()
example_dict

{'DS1|10451': Timestamp('2014-06-29 00:00:00')}

In [25]:
example_indexes

[Timestamp('2013-06-25 00:00:00'),
 Timestamp('2013-11-14 00:00:00'),
 Timestamp('2014-04-24 00:00:00'),
 Timestamp('2014-06-29 00:00:00')]